# 예제 1 : 뉴스 제목 + 링크 + 이미지 크롤링

In [2]:
'''
[필요한 라이브러리]
'''
import requests                      # 웹 요청을 보내는 라이브러리
from bs4 import BeautifulSoup       # HTML 파싱을 위한 라이브러리

In [4]:
# 크롤링 대상 URL (네이버 뉴스 > 생활/문화 섹션)
url = "https://news.naver.com/section/103"

In [6]:
# 브라우저처럼 보이기 위한 헤더 설정 (봇 차단 방지용)
headers = {
    "User-Agent" : "Mozilla/5.0"
}

In [8]:
# Http GET 요청을 보내기
res = requests.get(url, headers = headers)

In [10]:
# 받은 HTML 문서를 파싱하여 soup 객체 생성
soup = BeautifulSoup(res.text, "lxml")

In [12]:
'''
[soup.select("css 선택자")]

- html문서 안에서 css 선택자 문법을 써서 원하는 요소들을 리스트 형태로 가져옴
- 결과는 항상 리스트로 변환
'''

# 뉴스 제목과 링크가 포함된 'a' 태그들 선택 (class 속성에 'sa_text_title' 포함)
news_list = soup.select("a.sa_text_title")

In [61]:
'''
[각 뉴스 항목 순회]

- start = 1 : enumerate()는 인덱스를 0부터 시작하는데, start = 1 1부터 시작해 달라는 뜻
'''

for i, a_tag in enumerate(news_list, start = 1) :

    '''
    [뉴스 제목 텍스트 추출(공백 제거)]

    - tag.get_text(strip = False, True)
    - False라면 공백을 지워주지 않음
    - True라면 공백을 지준다.
    '''
    title = a_tag.get_text(strip = True)

    
    '''
    [뉴스 기사 링크 추출 (a 태그의 href 속성)]
    '''
    link = a_tag["href"]

    
    '''
    [뉴스 이미지 추출]

    - 현재 'a' 태그의 부모 요소 중 'li' 태그를 찾음
    - find_parent() : 특정 태그의 부모 태그를 찾아주는 기능 > 태그의 한 단계 위 부모 찾기
    - find_parents() : 태그의 모든 조상들 찾기(리스트)

    [li 태그 안에 있는 img 태그 찾기]

    - (있을 경우에만)
    - soup.select_one("img") : BeautifulSoup에서 사용하는 함수고, "처음 나오는 하나의 요소"만 선택
    - soup.select("img") -> 리스트로 가져온다.
    '''
    parent_li = a_tag.find_parent("li")

    img_tag = parent_li.select_one("img") if parent_li else None

    
    '''
    [뉴스 이미지 URL 추출]

    1. 이미지가 자연 로딩 방식이면 data-src 속성 사용
        - data-src : 자바스크립트나 크롤러가 데이터를 읽을 때 사용
        - html에서 사용자 정의 속성을 만들 때 사용
    2. 일반적인 경우에는 src 속성 사용
        - img_tag라는 이미지 태그가 존재하고, 그 태그에 src 속성이 있으면, 그 src 속성의 값을 img_url이라는 변수에 저장하라는 의미이다.
    3. 이미지가 없는 경우
    '''
    if img_tag and img_tag.has_attr("data-src") :
        img_url = img_tag["data-src"]
    elif img_tag and img_tag.has_attr("src") :
        img_url = img_tag["src"]
    else :
        img_url = "이미지 없음"

    # 결과 출력
    print(f"{i}. {title}")    # 번호와 뉴스 제목
    print(f"> {link}")        # 뉴스 상세 페이지 url
    print(f"> {img_url}")     # 이미지 url 또는 '이미지 없음'
    # 구분선
    print("---")

1. 서울 36.2℃, 불볕더위...일요일까지 서쪽 '펄펄'[날씨]
> https://n.news.naver.com/mnews/article/052/0002217202
> https://mimgnews.pstatic.net/image/origin/052/2025/07/10/2217202.jpg?type=nf220_150
---
2. '흥행 가도' 론 뮤익 전시, 90일 만에 50만 관객 돌파
> https://n.news.naver.com/mnews/article/055/0001274127
> https://mimgnews.pstatic.net/image/origin/055/2025/07/10/1274127.jpg?type=nf220_150
---
3. 끝날 줄 모르는 폭염…기세줄었지만 금요일에도 낮 최고 36도
> https://n.news.naver.com/mnews/article/001/0015500491
> https://mimgnews.pstatic.net/image/origin/001/2025/07/10/15500491.jpg?type=nf220_150
---
4. 주말까지 지금같은 '땡볕더위'…이후에도 폭염특보급 무더위
> https://n.news.naver.com/mnews/article/001/0015499326
> https://mimgnews.pstatic.net/image/origin/001/2025/07/10/15499326.jpg?type=nf220_150
---
5. [뉴스초점] 폭염 열돔에 갇힌 한반도…주말까지도 '펄펄'
> https://n.news.naver.com/mnews/article/422/0000758447
> https://mimgnews.pstatic.net/image/origin/422/2025/07/10/758447.jpg?type=nf220_150
---
6. [퇴근길 날씨] 연일 불볕더위…폭염 계속
> https://n.news.naver.com/mnews/article/056/0011986928
> htt

# 2. 예제 2 : 뉴스 제목 + 링크 + 이미지 크롤링 + 엑셀 저장

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [19]:
url = "https://news.naver.com/section/103"

In [21]:
headers = {
    "User-Agent" : "Mozilla/5.0"
}

In [23]:
res = requests.get(url, headers = headers)

In [25]:
soup = BeautifulSoup(res.text, "lxml")

In [27]:
news_list = soup.select("a.sa_text_title")

In [29]:
# 각 뉴스의 제목, 링크, 이미지 url을 저장 빈 리스트 생성
titles = []
links = []
images = []

In [31]:
for i, a_tag in enumerate(news_list, start = 1) :
    
    # 뉴스 제목 텍스트 추출 (공백 제거)
    title = a_tag.get_text(strip = True)

    # 뉴스 기사 링크 추출 (a 태그의 href 속성)
    link = a_tag["href"]

    # 뉴스 이미지 추출
    parent_li = a_tag.find_parent("li")

    img_tag = parent_li.select_one("img") if parent_li else None

    # 뉴스 이미지 URL 추출
    if img_tag and img_tag.has_attr("data-src") :
        img_url = img_tag["data-src"]
    elif img_tag and img_tag.has_attr("src") :
        img_url = img_tag["src"]
    else :
        img_url = "이미지 없음"

    # 각각의 리스트에 수집한 데이터를 추가
    titles.append(title)
    links.append(link)
    images.append(img_url)

# 수집한 데이터를 DataFrame(표 형태)으로 변환
df = pd.DataFrame({
    "제목" : titles,
    "링크" : links,
    "이미지" : images
})

# DataFrame을 엑셀 파일로 저장
df.to_excel("naver_news.xlsx", index=False)

# 저장 완료 메시지 출력
print("엑셀 저장 완료!")

엑셀 저장 완료!


# 예제 3 : 뉴스 본문 내용 + 요약 (w/정규 표현식)

In [34]:
'''
[정규 표현식]

- 문자나 기호로 만든 검색 패턴
- "특정 조건에 맞는 문자"를 찾고, 검사하고, 바꾸고, 추출할 수 있다.

[언제 사용할까?]
- 이메일 주소 뽑기, 전화번호 찾기, 문서에서 특정 문장만 걸러내기, 주문등록번호, 학번, 특정 포맷 감지
- 웹 크롤링 중 원하는 텍스트만 추출
'''

import re    # 정규 표현식을 다루기 위한 표준 라이브러리

In [36]:
r'''
[숫자 추출]

- \d "숫자 하나"  + -> 1개 이상 반복-즉, 숫자가 연속된 부분을 찾겠다.
- findall : 정규 표현식 p에 맞는 모든 값을 찾아서 리스트로 반환
'''

text = "오늘은 2025년 4월 22일입니다"

pattern = r"\d+"

numbers = re.findall(pattern, text)

print("숫자만 추출 ", numbers)

숫자만 추출  ['2025', '4', '22']


In [38]:
r'''
[이메일 주소 추출]

>>> \w+@\w+\.\w+

- \w+ : 영문자, 숫자, 밑줄 중 하나 이상(사용자 이름 또는 도메인 앞부분)
- @ : 골뱅이 문자 그대로 찾음
- \w+ : 골뱅이 뒤의 도메인 이름
- \. : 마침표를 의미함
- \w+ : com.net.org 같은 도메인 확장자 부분
'''

text = "문의는 contact@naver.com 또는 help@gmail.com 으로 주세요."

pattern = r"\w+@\w+\.\w+"

email = re.findall(pattern, text)

print("이메일 ", email)

이메일  ['contact@naver.com', 'help@gmail.com']


In [40]:
r'''
[한글만 추출]

- 한글로 이루어진 단어들만 뽑아내는 정규식
- [가-힣] : 유니코드 상 한글 완성형 글자 범위
- 영어 단어만 찾고 싶으면 [a-zA-Z]+
- 숫자만 찾고 싶으면 \d+
- 한글+영어 [가-힣a-zA-Z]
'''

text = "hello 안녕하세요 123"

pattern = r"[가-힣]+"

ko = re.findall(pattern, text)

print("한글 단어 ", ko)

한글 단어  ['안녕하세요']


# 예제 4 : 뉴스 본문 내용 + 요약

In [43]:
import requests
from bs4 import BeautifulSoup
import re

In [45]:
url = "https://n.news.naver.com/mnews/article/018/0005994547"

In [47]:
headers = {
    "User-Agent" : "Mozilla/5.0"
}

In [49]:
res = requests.get(url, headers = headers)

In [51]:
soup = BeautifulSoup(res.text, "html.parser")

In [53]:
# 기사 논문에 들어있는 <article> 태그 가져오기 (id = "dic_area")를 찾기
article = soup.find("article", id="dic_area")

In [55]:
# 기사 본문에서 태그를 제거하고 순수 텍스트만 추출
if article :
    # separator="\n" : 문단 구분을 위해 줄바꿈 문자로 구분
    text = article.get_text(strip=True, separator="\n")
    print("본문 : \n")
    print(text)
    print()
else :
    # 본문이 없을 경우 에러 메시지 출력
    print("본문을 찾을 수 없습니다.")

본문 : 

최저기온 8~16도, 최고기온 14~25도
충청권·남부지방은 한때 이슬비
"해안가 짙은 안개 발생해 주의 요구돼"
[이데일리 이영민 기자] 23일 수도권과 강원 영서 지역은 화창한 날씨가 나타나겠다. 충청권과 그 밖의 지역에는 한때 이슬비가 내리고 구름이 껴 흐리겠다.
전날 내린 비가 그치고 맑은 날씨를 보이는 지난 20일 서울 서초구 반포한강공원에서 시민들이 자전거를 타고 있다.(사진=연합뉴스)
22일 기상청에 따르면 이날 아침 최저기온은 8~16도, 낮 최고기온은 14~25도가 되겠다. 당분간 기온은 평년(최저기온 5~11도, 최고기온 18~22도)보다 2~4도가량 높을 전망이다. 기상청은 수도권과 강원 내륙, 충북을 중심으로 낮과 밤의 기온 차이가 15도 내외로 크게 벌어져 건강관리에 유의해야 한다고 당부했다.
앞서 내린 비의 영향으로 충청권과 그 밖의 남부지방은 이날 오전까지, 전라 동부지역과 경남 내륙, 제주도에는 오후 6시까지 0.1㎜ 미만의 빗방울이 떨어지는 곳이 있겠다.
풍랑특보가 발효된 서해 남부 먼바다는 새벽 시간대까지 바람이 강하게 불겠고 물결이 1.5~4m 높이로 매우 높게 일 수 있다. 동해 남부 북쪽 먼바다와 동해 앞바다에도 바람이 차차 강하게 불고 물결이 높게 일면서 풍랑특보가 발표될 가능성이 있다.



In [57]:
r'''
[정규 표현식을 사용해 문장을 분리]

>>> (?<=[.!?])\s+

- 마침표. 느낌표! 물음표? 뒤에 공백이 올 경우를 기준으로 분할
'''
sentences = re.split(r"(?<=[.!?])\s+", text)

In [59]:
'''
[상위 3~5개의 문장만 뽑아서 요약 형태로 출력]

* sentences[:5] : 앞에서부터 최대 5개 문장만 추출

[문장 해석]

- sentences 리스트에서 앞의 5개 문장만 가져와서, 각 문장 사이를 줄바꿈으로 연결해서 하나의 문단처럼 만든 문자열을 summary에 저장한다.
'''
summary = "\n".join(sentences[:5])

# 요약된 문단 출력
print("요약 : \n")
print(summary)
print()

요약 : 

최저기온 8~16도, 최고기온 14~25도
충청권·남부지방은 한때 이슬비
"해안가 짙은 안개 발생해 주의 요구돼"
[이데일리 이영민 기자] 23일 수도권과 강원 영서 지역은 화창한 날씨가 나타나겠다.
충청권과 그 밖의 지역에는 한때 이슬비가 내리고 구름이 껴 흐리겠다.
전날 내린 비가 그치고 맑은 날씨를 보이는 지난 20일 서울 서초구 반포한강공원에서 시민들이 자전거를 타고 있다.(사진=연합뉴스)
22일 기상청에 따르면 이날 아침 최저기온은 8~16도, 낮 최고기온은 14~25도가 되겠다.
당분간 기온은 평년(최저기온 5~11도, 최고기온 18~22도)보다 2~4도가량 높을 전망이다.
기상청은 수도권과 강원 내륙, 충북을 중심으로 낮과 밤의 기온 차이가 15도 내외로 크게 벌어져 건강관리에 유의해야 한다고 당부했다.

